In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: alinatl to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
%%capture
!pip install emoji
!pip install evaluate
!pip install rouge_score

In [ ]:
from transformers.optimization import Adafactor
from transformers import get_constant_schedule_with_warmup
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from typing import Union
from transformers import EvalPrediction

In [1]:
import zipfile
import os
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
import torch

In [ ]:
!unzip /content/short_kloop.zip
with open('/content/short_kloop.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines[:10]:
    print(line.strip())

Эгер ал да жардам бербесе, тематикалык форумдарга барыңыз.
29-июнда президент Садыр Жапаров УКМКга караштуу «Айкөл» балдардын ден соолугун чыңдоо борборун ачты.
Жусубалиев Балыкчы-Корумду жолун бүтүрүш үчүн 5,3 миллиард сом зарыл экенин билдирди.
Депутат Бахридин Шабазов күчтүү таасир берүүчү заттарды колдонууга көндүрүү үчүн кылмыш жоопкерчилигин киргизүүнү сунуштады.
Кыргызстандыктар 2022-жылы TikTok’то атагын чыгарып, миллиондогон көрүүлөрдү топтоп, дүйнөлүк деңгээлге чыгышты.
Kloop.kgнин редакциясы Орто Азия өлкөлөрүнүн интернет таркатуучуларынын сырткы трафикти алуусу жөнүндөгү жеке маалыматтарына таянуу менен, эгер Орусия тосмологондо Казакстан, Өзбекстан жана Тажикстандын колдонуучулары да «Фэйсбукка» кирүү мүмкүнчүлүктөрүнөн ажырашмак деп түшүндү.
Кыргызстанда финалга чыккан биринчи аял болдум.
Аларга ыңгайлуу бааны сунуштабасаң кайра кайтып келбей коюшат.
Ал жердеакча көп.
Текебаев менен Чотонов акталды.


## Tokenization

In [ ]:
from collections import Counter

chars_cnt = Counter(c for t in lines for c in t)
required_chars = ''.join([
    k for k, v in chars_cnt.most_common()
    if v >= 3 and k not in ' '
])

import sentencepiece as spm
text_file = '/content/short_kloop.txt'
SPM_PREFIX = 'spm_kyr_16k'

spm.SentencePieceTrainer.train(
    input=text_file,
    model_prefix=SPM_PREFIX,
    vocab_size=2**14,
    character_coverage = 1,
    num_threads=16,
    train_extremely_large_corpus=False,
    add_dummy_prefix=False,
    max_sentencepiece_length=128,
    max_sentence_length=4192*4,
    pad_id=0,
    eos_id=1,
    unk_id=2,
    bos_id=-1,
    required_chars=required_chars,
)

In [ ]:
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
from transformers import M2M100Tokenizer, M2M100Config

tokenizer = M2M100Tokenizer.from_pretrained('facebook/m2m100_418M')
sp_trained = spm.SentencePieceProcessor(model_file=f'{SPM_PREFIX}.model')
added_spm = sp_pb2_model.ModelProto()
added_spm.ParseFromString(sp_trained.serialized_model_proto())
old_spm = sp_pb2_model.ModelProto()
old_spm.ParseFromString(tokenizer.sp_model.serialized_model_proto())

tokens_set = {p.piece for p in old_spm.pieces}

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from tqdm import tqdm
model_name = 'facebook/m2m100_418M'

tokenizer_old = M2M100Tokenizer.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

new_tokens = set({p.piece for p in added_spm.pieces}) -  set(tokens_set)
tokenizer.add_tokens(list(new_tokens))

print(len(tokenizer_old), len(tokenizer))
added_vocab = set(tokenizer.get_vocab()).difference(set(tokenizer_old.get_vocab()))
print(len(added_vocab))

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

for t in tqdm(added_vocab):
    tt = tokenizer_old(t, add_special_tokens=False).input_ids
    if len(tt) == 0:
        tt = [tokenizer_old.unk_token_id]
    idx = tokenizer.convert_tokens_to_ids(t)
    model.model.shared.weight.data[idx] = model.model.shared.weight.data[tt].mean(0)

128104 142570
14466


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
100%|██████████| 14466/14466 [00:10<00:00, 1352.66it/s]


In [ ]:
tokenizer.src_lang = 'en'
tokenizer(['My name is alina'], return_special_tokens_mask=True), tokenizer.src_lang

({'input_ids': [[128022, 6384, 33969, 117, 95342, 2]], 'special_tokens_mask': [[1, 0, 0, 0, 0, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]},
 'en')

In [ ]:
new_lang_code = "ky"
similar_lang_code = "__kk__"
additional_special_tokens = {'additional_special_tokens' : tokenizer.additional_special_tokens + ['__ky__']}
num_added_tokens = tokenizer.add_special_tokens(additional_special_tokens, replace_additional_special_tokens=True)

new_lang_id = tokenizer.convert_tokens_to_ids('__ky__')
similar_lang_id = tokenizer.convert_tokens_to_ids(similar_lang_code)
print(new_lang_id, similar_lang_id)

tokenizer.lang_code_to_id[new_lang_code] = new_lang_id
tokenizer.lang_code_to_token["ky"] = "__ky__"
tokenizer.lang_token_to_id["__ky__"] = tokenizer.convert_tokens_to_ids("__ky__")

model.resize_token_embeddings(len(tokenizer))
with torch.no_grad():
    model.model.shared.weight[new_lang_id] = model.model.shared.weight[similar_lang_id].clone()
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

142570 128049


In [ ]:
print(f"New language '__ky__' ID: {tokenizer.convert_tokens_to_ids('__ky__')}")
print(f"'__ky__' in additional_special_tokens: {'__ky__' in tokenizer.additional_special_tokens}")
if hasattr(tokenizer, 'lang_code_to_id'):
    print(f"'{new_lang_code}' in lang_code_to_id: {new_lang_code in tokenizer.lang_code_to_id}")
else:
    print("Note: This tokenizer doesn't have a 'lang_code_to_id' attribute.")

New language '__ky__' ID: 142570
'__ky__' in additional_special_tokens: True
'ky' in lang_code_to_id: True


In [ ]:
tokenizer.src_lang = 'ky'
tokenizer.src_lang, tokenizer.additional_special_tokens, tokenizer.lang_code_to_id
tokenizer.convert_tokens_to_ids('__ky__'), tokenizer(['My name is alina'], return_special_tokens_mask=True),

(142570,
 {'input_ids': [[142570, 6384, 33969, 117, 95342, 2]], 'special_tokens_mask': [[1, 0, 0, 0, 0, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]})

In [ ]:
tokenizer.src_lang = 'ky'
tokenizer.tgt_lang = 'en'

batch = tokenizer(['Менин атым Алина.'], return_tensors='pt', padding=True, truncation=True, max_length=64)
print(batch)
input_ids = batch['input_ids']
print(tokenizer.convert_ids_to_tokens(input_ids[0]))
attention_mask = batch['attention_mask']

labels = tokenizer(['My name is Alina.'], return_tensors='pt', padding=True, truncation=True, max_length=64)['input_ids']
labels[labels == tokenizer.pad_token_id] = -100
print(tokenizer.convert_ids_to_tokens(labels[0]))

outputs_2 = model.generate(input_ids=input_ids, forced_bos_token_id=tokenizer.get_lang_id("en"), max_length=30) # decoder_start_token_id=tokenizer.eos_token_id, # forced_bos_token_id=tokenizer.convert_tokens_to_ids("ky"),
decoded_preds = tokenizer.batch_decode(outputs_2, skip_special_tokens=True)
print(decoded_preds)
print(tokenizer.convert_ids_to_tokens(outputs_2[0]))

{'input_ids': tensor([[142570, 140052,   7027,    327,  23376,    532,  21072,      5,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}
['__ky__', 'Мени', '▁н', '▁а', 'тым', '▁А', 'лина', '.', '</s>']
['__ky__', '▁My', '▁name', '▁is', '▁Al', 'ina', '.', '</s>']
['I am Alina.']
['</s>', '__en__', '▁I', '▁am', '▁Al', 'ina', '.', '</s>']


## inference

In [ ]:
max_seq_len = 128
model.eval()
model.to('cuda')

example_input_str = ['Менин атым Алина!',
                     'Менин сүйүктүү жазуучум – Чыңгыз Айтматов.',
                     'Кандайсыз?']

tokenizer.src_lang = 'ky'
tokenizer.tgt_lang = 'en'
for sent in example_input_str:
    print(sent)
    token_ids = tokenizer([sent], return_tensors='pt', padding=True, truncation=True, max_length=max_seq_len)
    input_ids = token_ids['input_ids']
    attention_mask = token_ids['attention_mask']

    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids.to('cuda'), forced_bos_token_id=tokenizer.get_lang_id("en"), max_length=max_seq_len)
        decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        print(tokenizer.convert_ids_to_tokens(generated_ids[0]))
        print(decoded_preds)

Менин атым Алина!
['</s>', '__en__', '▁I', '▁am', '▁Al', 'ina', '!', '</s>']
['I am Alina!']
Менин сүйүктүү жазуучум – Чыңгыз Айтматов.
['</s>', '__en__', '▁An', 'ish', 'in', 'abe', '▁aking', '▁ate', '▁aw', 'aso', '▁kik', 'ina', 'wad', 'jich', 'igan', '.', '</s>']
['Anishinabe aking ate awaso kikinawadjichigan.']
Кандайсыз?
['</s>', '__en__', '▁What', '▁is', '▁it', '?', '</s>']
['What is it?']


## load data

In [2]:
from datasets import load_dataset

kir = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="https://huggingface.co/datasets/openlanguagedata/oldi_seed/resolve/refs/pr/4/seed/kir_Cyrl.parquet"
)['train']

eng = load_dataset(
    "openlanguagedata/oldi_seed",
    data_files="seed/eng_Latn.parquet"
)["train"]

eng_dict = {row["id"]: row["text"] for row in eng}
merged = kir.map(lambda row: {"kir": row["text"], "eng": eng_dict.get(row["id"], None)})
train_dataset = merged.select_columns(["eng", "kir"])

print(train_dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

seed/kir_Cyrl.parquet:   0%|          | 0.00/884k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

seed/eng_Latn.parquet:   0%|          | 0.00/601k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6193 [00:00<?, ? examples/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 6193
})


In [ ]:
eng_ds = load_dataset("openlanguagedata/flores_plus", "eng_Latn", split="dev")
kir_ds = load_dataset("openlanguagedata/flores_plus", "kir_Cyrl", split="dev")

assert len(eng_ds) == len(kir_ds)

parallel_data = [
    {"eng": eng_ds[i]["text"], "kir": kir_ds[i]["text"]}
    for i in range(len(eng_ds))
]

test_dataset = Dataset.from_list(parallel_data)
test_dataset

Resolving data files:   0%|          | 0/220 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/214 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/220 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/214 [00:00<?, ?it/s]

Dataset({
    features: ['eng', 'kir'],
    num_rows: 997
})

In [ ]:
from torch.utils.data import DataLoader, Dataset

def tokenize_data(dataset, source_col='kir', target_col='eng', tokenizer=None, max_length=128):
    tokenizer.src_lang = 'ky'
    source_tokenized = tokenizer(
        list(dataset[source_col]),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length
    )
    tokenizer.src_lang = 'en'
    target_tokenized = tokenizer(
        list(dataset[target_col]),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=max_length
    )
    return source_tokenized, target_tokenized


class TranslationDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }
max_length = 128
source_tokenized_train, target_tokenized_train = tokenize_data(train_dataset, tokenizer=tokenizer, max_length=max_length)
source_tokenized_test, target_tokenized_test = tokenize_data(test_dataset, tokenizer=tokenizer, max_length=max_length)

input_ids_train = source_tokenized_train['input_ids']
attention_mask_train = source_tokenized_train['attention_mask']
labels_train = target_tokenized_train['input_ids']

input_ids_test = source_tokenized_test['input_ids']
attention_mask_test = source_tokenized_test['attention_mask']
labels_test = target_tokenized_test['input_ids']

print("Train input_ids shape:", input_ids_train.shape)
print("Train attention_mask shape:", attention_mask_train.shape)
print("Train labels shape:", labels_train.shape)

print("Test input_ids shape:", input_ids_test.shape)
print("Test attention_mask shape:", attention_mask_test.shape)
print("Test labels shape:", labels_test.shape)

train_dataset = TranslationDataset(input_ids_train, attention_mask_train, labels_train) # Access the input_ids from tokenized labels
test_dataset = TranslationDataset(input_ids_test, attention_mask_test, labels_test)  # Access the input_ids from tokenized labels

Train input_ids shape: torch.Size([6193, 128])
Train attention_mask shape: torch.Size([6193, 128])
Train labels shape: torch.Size([6193, 127])
Test input_ids shape: torch.Size([997, 112])
Test attention_mask shape: torch.Size([997, 112])
Test labels shape: torch.Size([997, 83])


## train

In [ ]:
import gc
import torch

def cleanup():
    """ A function to clean memory (mostly to prevent GPU memory leaks) """
    gc.collect()
    torch.cuda.empty_cache()
cleanup()

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
chrfpp = evaluate.load("chrf")

In [ ]:
def compute_metrics(eval_pred: Union[EvalPrediction, tuple], compute_result=False):

    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    if isinstance(predictions, torch.Tensor):
        predictions = predictions.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    if predictions.ndim == 3:
        predictions = np.argmax(predictions, axis=-1)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_predictions = [pred.strip() for pred in decoded_predictions]
    decoded_labels = [label.strip() for label in decoded_labels]

    bleu.add_batch(predictions=decoded_predictions, references=decoded_labels)
    chrfpp.add_batch(predictions=decoded_predictions, references=decoded_labels)

    if compute_result:
        bleu_result = bleu.compute()
        chrfpp_result = chrfpp.compute()

        print(f"CHRF++: {chrfpp_result['score']:.4f} | BLEU: {bleu_result['bleu']:.4f}")

        return {
            "chrf++": chrfpp_result['score'],
            "bleu": bleu_result["bleu"],
        }

    return {}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"/content/m2m100-ky-en",
    num_train_epochs=6,
    eval_strategy='steps',
    logging_steps=90,
    eval_steps = 90,
    torch_empty_cache_steps=90,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_accumulation_steps=4,
    gradient_accumulation_steps=2,
    warmup_steps=10,
    weight_decay=1e-3,
    optim="adamw_torch",
    save_total_limit=4,
    learning_rate=1e-4,
    batch_eval_metrics=True,
    report_to="wandb",
    save_strategy="no",
    push_to_hub=True,
    hub_model_id="alinatl/m2m100-ky-en",
    fp16=True,
)

In [ ]:
len(train_dataset) / (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * 1

193.53125

In [ ]:
model.generation_config.forced_bos_token_id = tokenizer.get_lang_id("en")
model.generation_config.decoder_start_token_id=tokenizer.eos_token_id
model.generation_config.max_length=max_length
model.generation_config

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "early_stopping": true,
  "eos_token_id": 2,
  "forced_bos_token_id": 128022,
  "max_length": 128,
  "num_beams": 5,
  "pad_token_id": 1
}

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.init(project='NMT-ky-en')
trainer.train(resume_from_checkpoint=False)

/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Chrf++,Bleu
90,3.453000,1.417120,31.540558,0.061919
180,0.779300,1.246251,36.729591,0.091204
270,0.557300,1.209596,38.616686,0.105914
360,0.506000,1.148090,39.957715,0.118649
450,0.396400,1.147917,40.856860,0.125798
540,0.354300,1.137159,41.342377,0.126147
630,0.288600,1.138661,42.253148,0.135165
720,0.255400,1.134381,42.022659,0.134824
810,0.232100,1.131855,42.287561,0.136030
900,0.192100,1.136503,42.709110,0.137402


CHRF++: 31.5406 | BLEU: 0.0619
CHRF++: 36.7296 | BLEU: 0.0912
CHRF++: 38.6167 | BLEU: 0.1059
CHRF++: 39.9577 | BLEU: 0.1186
CHRF++: 40.8569 | BLEU: 0.1258
CHRF++: 41.3424 | BLEU: 0.1261
CHRF++: 42.2531 | BLEU: 0.1352
CHRF++: 42.0227 | BLEU: 0.1348
CHRF++: 42.2876 | BLEU: 0.1360
CHRF++: 42.7091 | BLEU: 0.1374
CHRF++: 42.5069 | BLEU: 0.1374
CHRF++: 42.9292 | BLEU: 0.1388


TrainOutput(global_step=1164, training_loss=0.5791690308613466, metrics={'train_runtime': 2279.7956, 'train_samples_per_second': 16.299, 'train_steps_per_second': 0.511, 'total_flos': 1.0065659100659712e+16, 'train_loss': 0.5791690308613466, 'epoch': 6.0})

In [ ]:
!pip install push_to_hub

ERROR: Could not find a version that satisfies the requirement push_to_hub (from versions: none)
ERROR: No matching distribution found for push_to_hub


In [ ]:
trainer.push_to_hub('alinatl/m2m100-ky-en')

/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...2m100-ky-en/sentencepiece.bpe.model: 100%|##########| 2.42MB / 2.42MB            

  ...tent/m2m100-ky-en/model.safetensors:   0%|          | 27.0kB / 1.99GB            

  ...tent/m2m100-ky-en/training_args.bin:   8%|7         |   456B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/alinatl/m2m100-ky-en/commit/b58e6ac487460ea01f333dd9bb9150c7fd2ec57b', commit_message='alinatl/m2m100-ky-en', commit_description='', oid='b58e6ac487460ea01f333dd9bb9150c7fd2ec57b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alinatl/m2m100-ky-en', endpoint='https://huggingface.co', repo_type='model', repo_id='alinatl/m2m100-ky-en'), pr_revision=None, pr_num=None)